In [1]:
import os
os.listdir()

['.ipynb_checkpoints',
 'cluster-wise-dfs.pickle',
 'df-new-features.pickle',
 'exp-wa-past-values-dfs.pickle',
 'exp-wa-past-values-week-days-dfs.pickle',
 'feature extraction for regression.ipynb',
 'feb-2016-groupby.csv',
 'feb-2016-outliers-removed.csv',
 'feb-2016-smooth.pickle',
 'jan-2016-groupby.csv',
 'jan-2016-outliers-removed.csv',
 'jan-2016-smooth.pickle',
 'mar-2016-groupby.csv',
 'mar-2016-outliers-removed.csv',
 'mar-2016-smooth.pickle',
 'march-2016-groupby.csv',
 'march-2016-outliers-removed.csv',
 'mydask.png',
 'NYC Final.ipynb',
 'Regression models.ipynb',
 'rf-fi.pickle']

In [11]:
import pandas as pd
import pickle
import numpy as np

In [13]:
with open('jan-2016-smooth.pickle', 'rb') as f:
    jan_2016_smooth=pickle.load(f)
    
with open('feb-2016-smooth.pickle', 'rb') as f:
    feb_2016_smooth=pickle.load(f)
    
with open('mar-2016-smooth.pickle', 'rb') as f:
    mar_2016_smooth=pickle.load(f)

In [4]:
len(jan_2016_smooth)==40*(24*31*6)

True

In [5]:
(24*31*6)

4464

In [6]:
import math

In [21]:
def EA_P1_Predictions(ratios, alpha):
    predicted_value= (ratios['Prediction'].values)[0]*alpha 
    error=[]
    predicted_values=[]
    
    for i in range(0,13104*40):
        if i%13104==0:
            predicted_values.append(0)
            error.append(0)
            continue
        predicted_values.append(predicted_value)
        error.append(abs((math.pow(predicted_value-(ratios['Prediction'].values)[i],1))))
        predicted_value =int((alpha*predicted_value) + (1-alpha)*((ratios['Prediction'].values)[i]))
    
    ratios['EA_P1_Predicted'] = predicted_values
    ratios['EA_P1_Error'] = error
    mape_err = (sum(error)/len(error))/(sum(ratios['Prediction'].values)/len(ratios['Prediction'].values))
    mse_err = sum([e**2 for e in error])/len(error)
    return ratios,mape_err,mse_err

In [15]:
# combine time series of 3 months to generate one time series spread over 3 months for each of 40 regions

jan_bins=31*24*6
feb_bins=29*24*6
mar_bins=31*24*6

In [16]:
regions=[]
for r in range(40):
    regions.append(jan_2016_smooth[r*jan_bins:(r+1)*jan_bins]+feb_2016_smooth[r*feb_bins:(r+1)*feb_bins]+
                   mar_2016_smooth[r*mar_bins:(r+1)*mar_bins])

In [17]:
total_bins=jan_bins+feb_bins+mar_bins

In [18]:
# jan_bins+feb_bins+mar_bins==13104
import numpy as np
np.all(np.array([len(r) for r in regions])==total_bins)

True

In [19]:
combined_pickups=[]
for r in regions:
    combined_pickups.extend(r)

In [22]:
ratios=pd.DataFrame()
ratios['Prediction']=combined_pickups
ratios, mape_err, mse_err=EA_P1_Predictions(ratios, 0.3)

In [42]:
mape_err # error reduces as data increases. Using only jan 2016 data gives 13.5% error

0.13191766220001513

In [48]:
# region wise division of dataframes
regions_df=[]
for r in range(40):
    regions_df.append(ratios[r*total_bins:(r+1)*total_bins])

In [49]:
regions_df[0].head() # 3 month time series prediction for region 0 using exponential weighted averages of past values

,Prediction,EA_P1_Predicted,EA_P1_Error
0,0,0.0,0.0
1,63,0.0,63.0
2,217,44.0,173.0
3,189,165.0,24.0
4,137,181.0,44.0


In [50]:
regions_df[1].head() # 3 month time series prediction for region 1 using exponential weighted averages of past values

,Prediction,EA_P1_Predicted,EA_P1_Error
13104,0,0.0,0.0
13105,0,45.0,45.0
13106,1,13.0,12.0
13107,0,4.0,4.0
13108,3,1.0,2.0


In [51]:
regions_df[2].head() # 3 month time series prediction for region 2 using exponential weighted averages of past values

,Prediction,EA_P1_Predicted,EA_P1_Error
26208,0,0.0,0.0
26209,119,0.0,119.0
26210,181,83.0,98.0
26211,180,151.0,29.0
26212,186,171.0,15.0


In [52]:
with open('exp-wa-past-values-dfs.pickle', 'wb') as f:
    pickle.dump(regions_df, f)

In [7]:
with open('exp-wa-past-values-dfs.pickle', 'rb') as f:
    regions_df=pickle.load(f)

In [9]:
def get_mape(predictions, truth):
    return 100*np.sum(np.abs(predictions-truth))/np.sum(truth)

In [56]:
jan_df=pd.read_csv('jan-2016-outliers-removed.csv')
feb_df=pd.read_csv('feb-2016-outliers-removed.csv')
mar_df=pd.read_csv('march-2016-outliers-removed.csv')

In [78]:
# generate 40 dataframes, one for each cluster
regions_df_full=[]

for r in range(40):
    regions_df_full.append(pd.concat((jan_df[jan_df['pickup_cluster']==r].sort_values(by='pickup_bins'), 
                                      feb_df[feb_df['pickup_cluster']==r].sort_values(by='pickup_bins'),
                                     mar_df[mar_df['pickup_cluster']==r].sort_values(by='pickup_bins'))))

In [104]:
with open('cluster-wise-dfs.pickle', 'wb') as f:
    pickle.dump(regions_df_full, f)

## One hot encoding of days of week

In [33]:
week_day=[]
for r in range(40):
    week_day.append([int(((int(k/144))%7+4)%7) for k in range(total_bins)])

In [34]:
for r in range(40):
    regions_df[r]['week_day']=week_day[r]

In [103]:
with open('exp-wa-past-values-week-days-dfs.pickle', 'wb') as f:
    pickle.dump(regions_df, f)

In [35]:
with open('exp-wa-past-values-week-days-dfs.pickle', 'rb') as f:
    regions_df=pickle.load(f)

In [36]:
from sklearn.preprocessing import OneHotEncoder

In [37]:
def one_hot_encode(df, col):
    enc = OneHotEncoder(categories='auto')
    enc.fit(np.array([df[col].values]).T)
    
    col_one_hot_encoded=np.array(enc.transform(np.array([df[col].values]).T).todense())
#     df.drop(col, axis=1, inplace=True)
    
    for i, category in enumerate(enc.categories_[0]):
        df[str(category)]=col_one_hot_encoded[:, i]

In [38]:
for df in regions_df:
    one_hot_encode(df, 'week_day')
    df.drop('week_day', axis=1, inplace=True)

In [39]:
regions_df[0].columns

Index(['Prediction', 'EA_P1_Predicted', 'EA_P1_Error', '0', '1', '2', '3', '4',
       '5', '6'],
      dtype='object')

In [73]:
regions_df[1].columns

Index(['Prediction', 'EA_P1_Predicted', 'EA_P1_Error', '0', '1', '2', '3', '4',
       '5', '6'],
      dtype='object')

In [74]:
regions_df[-1].columns

Index(['Prediction', 'EA_P1_Predicted', 'EA_P1_Error', '0', '1', '2', '3', '4',
       '5', '6'],
      dtype='object')

## Pickup bin value 1 week earlier

In [40]:
len(regions_df[0])

13104

In [41]:
print(f'no of rows lost = {7*24*6}, remaining rows = {13104-7*24*6}')

no of rows lost = 1008, remaining rows = 12096


In [42]:
for df in regions_df:
    df['last_week_pred']=df['Prediction'].shift(7*24*6)

In [43]:
regions_df[0].iloc[7*24*6]

Prediction         40.0
EA_P1_Predicted    52.0
EA_P1_Error        12.0
0                   0.0
1                   0.0
2                   0.0
3                   0.0
4                   1.0
5                   0.0
6                   0.0
last_week_pred      0.0
Name: 1008, dtype: float64

## Pickup bin value 1 day earlier

In [44]:
for df in regions_df:
    df['previous_day_pred']=df['Prediction'].shift(24*6)

In [45]:
regions_df[0].columns

Index(['Prediction', 'EA_P1_Predicted', 'EA_P1_Error', '0', '1', '2', '3', '4',
       '5', '6', 'last_week_pred', 'previous_day_pred'],
      dtype='object')

## Last 5 pickup bin values

In [46]:
for df in regions_df:
    for prev in range(1,6):
        df['p_'+str(prev)]=df['Prediction'].shift(prev)

In [47]:
regions_df[0].columns

Index(['Prediction', 'EA_P1_Predicted', 'EA_P1_Error', '0', '1', '2', '3', '4',
       '5', '6', 'last_week_pred', 'previous_day_pred', 'p_1', 'p_2', 'p_3',
       'p_4', 'p_5'],
      dtype='object')

## Max of last 5 values

In [48]:
for df in regions_df:
    df['max_last_5']=df[['p_1', 'p_2', 'p_3', 'p_4', 'p_5']].apply(max, axis=1)

In [49]:
regions_df[0].columns

Index(['Prediction', 'EA_P1_Predicted', 'EA_P1_Error', '0', '1', '2', '3', '4',
       '5', '6', 'last_week_pred', 'previous_day_pred', 'p_1', 'p_2', 'p_3',
       'p_4', 'p_5', 'max_last_5'],
      dtype='object')

## Min of last 5 values

In [50]:
for df in regions_df:
    df['min_last_5']=df[['p_1', 'p_2', 'p_3', 'p_4', 'p_5']].apply(min, axis=1)

In [51]:
regions_df[0].columns

Index(['Prediction', 'EA_P1_Predicted', 'EA_P1_Error', '0', '1', '2', '3', '4',
       '5', '6', 'last_week_pred', 'previous_day_pred', 'p_1', 'p_2', 'p_3',
       'p_4', 'p_5', 'max_last_5', 'min_last_5'],
      dtype='object')

## Std dev of last 10 values

In [52]:
def std_dev(numbers):
    return np.var(numbers)**0.5

In [53]:
for df in regions_df:
    col_names=['p_'+str(i) for i in range(1,11)]
    for shift in range(6, 11):
        df['p_'+str(shift)]=df['Prediction'].shift(shift)
        
    df['std_dev_10']=df[col_names].apply(std_dev, axis=1)
    
    for shift in range(6, 11):
        df.drop('p_'+str(shift), axis=1, inplace=True)

In [54]:
regions_df[0].columns

Index(['Prediction', 'EA_P1_Predicted', 'EA_P1_Error', '0', '1', '2', '3', '4',
       '5', '6', 'last_week_pred', 'previous_day_pred', 'p_1', 'p_2', 'p_3',
       'p_4', 'p_5', 'max_last_5', 'min_last_5', 'std_dev_10'],
      dtype='object')

In [55]:
regions_df[0].iloc[10]

Prediction           131.00000
EA_P1_Predicted      153.00000
EA_P1_Error           22.00000
0                      0.00000
1                      0.00000
2                      0.00000
3                      0.00000
4                      1.00000
5                      0.00000
6                      0.00000
last_week_pred             NaN
previous_day_pred          NaN
p_1                  152.00000
p_2                  164.00000
p_3                  150.00000
p_4                  129.00000
p_5                  135.00000
max_last_5           164.00000
min_last_5           129.00000
std_dev_10            58.67231
Name: 10, dtype: float64

## DFT of last 2 hour values

In [56]:
def dft(series):
    return np.abs(np.fft.fft(series))

In [57]:
for df in regions_df:
    col_names=['p_'+str(i) for i in range(1,13)]
    for shift in range(6, 13):
        df['p_'+str(shift)]=df['Prediction'].shift(shift)
        
    dft_series=df[col_names].apply(dft, axis=1).values
    
    for shift in range(1, 13):
        component=[]
        for arr in dft_series:
            component.append(arr[shift-1])
        
        df['component_'+str(shift)]=component
        
        if shift>=6:
            df.drop('p_'+str(shift), axis=1, inplace=True)

In [58]:
regions_df[0].columns

Index(['Prediction', 'EA_P1_Predicted', 'EA_P1_Error', '0', '1', '2', '3', '4',
       '5', '6', 'last_week_pred', 'previous_day_pred', 'p_1', 'p_2', 'p_3',
       'p_4', 'p_5', 'max_last_5', 'min_last_5', 'std_dev_10', 'component_1',
       'component_2', 'component_3', 'component_4', 'component_5',
       'component_6', 'component_7', 'component_8', 'component_9',
       'component_10', 'component_11', 'component_12'],
      dtype='object')

In [59]:
with open('nans-included-all-features.pickle', 'wb') as f:
    pickle.dump(regions_df, f)

In [6]:
for df in regions_df:
    df.dropna(inplace=True)

In [8]:
len(regions_df[0])

12096

In [9]:
with open('df-new-features.pickle', 'wb') as f:
    pickle.dump(regions_df, f)